In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import functions as F

In [4]:
spark = SparkSession.builder \
.appName("filter") \
.master("local[2]") \
.getOrCreate()

# 1.1 `order_items` tablosunda kaç tane **tekil** `orderItemOrderId` var

In [5]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/order_items.csv

--2023-03-01 01:45:26--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/order_items.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5408988 (5.2M) [text/plain]
Saving to: ‘/home/train/datasets/order_items.csv’

100%[======================================>] 5,408,988   2.72MB/s   in 1.9s   

2023-03-01 01:45:29 (2.72 MB/s) - ‘/home/train/datasets/order_items.csv’ saved [5408988/5408988]



In [6]:
df_order_items = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/order_items.csv")

In [7]:
df_order_items.show()

+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|
+-------------+----------------+------------------+-----------------+-----------------+---------------------+
|            1|               1|               957|                1|           299.98|               299.98|
|            2|               2|              1073|                1|           199.99|               199.99|
|            3|               2|               502|                5|            250.0|                 50.0|
|            4|               2|               403|                1|           129.99|               129.99|
|            5|               4|               897|                2|            49.98|                24.99|
|            6|               4|               365|                5|           299.95|                59.99|
|         

In [8]:
from pyspark.sql.functions import countDistinct
df2=df_order_items.select(countDistinct("orderItemOrderId"))
df2.show()

+--------------------------------+
|count(DISTINCT orderItemOrderId)|
+--------------------------------+
|                           57431|
+--------------------------------+



# 1.2`orders` ve `order_items` tablolarında kaç satır var

In [10]:
df_order_items.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172198 entries, 0 to 172197
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   orderItemName          172198 non-null  int32  
 1   orderItemOrderId       172198 non-null  int32  
 2   orderItemProductId     172198 non-null  int32  
 3   orderItemQuantity      172198 non-null  int32  
 4   orderItemSubTotal      172198 non-null  float64
 5   orderItemProductPrice  172198 non-null  float64
dtypes: float64(2), int32(4)
memory usage: 5.3 MB


In [11]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/orders.csv

--2023-03-01 01:55:59--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/orders.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2999990 (2.9M) [text/plain]
Saving to: ‘/home/train/datasets/orders.csv’

100%[======================================>] 2,999,990   2.93MB/s   in 1.0s   

2023-03-01 01:56:01 (2.93 MB/s) - ‘/home/train/datasets/orders.csv’ saved [2999990/2999990]



In [12]:
df_orders = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/orders.csv")

In [13]:
df_orders.show()

+-------+--------------------+---------------+---------------+
|orderId|           orderDate|orderCustomerId|    orderStatus|
+-------+--------------------+---------------+---------------+
|      1|2013-07-25 00:00:...|          11599|         CLOSED|
|      2|2013-07-25 00:00:...|            256|PENDING_PAYMENT|
|      3|2013-07-25 00:00:...|          12111|       COMPLETE|
|      4|2013-07-25 00:00:...|           8827|         CLOSED|
|      5|2013-07-25 00:00:...|          11318|       COMPLETE|
|      6|2013-07-25 00:00:...|           7130|       COMPLETE|
|      7|2013-07-25 00:00:...|           4530|       COMPLETE|
|      8|2013-07-25 00:00:...|           2911|     PROCESSING|
|      9|2013-07-25 00:00:...|           5657|PENDING_PAYMENT|
|     10|2013-07-25 00:00:...|           5648|PENDING_PAYMENT|
|     11|2013-07-25 00:00:...|            918| PAYMENT_REVIEW|
|     12|2013-07-25 00:00:...|           1837|         CLOSED|
|     13|2013-07-25 00:00:...|           9149|PENDING_P

In [14]:
df_orders.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68883 entries, 0 to 68882
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   orderId          68883 non-null  int32 
 1   orderDate        68883 non-null  object
 2   orderCustomerId  68883 non-null  int32 
 3   orderStatus      68883 non-null  object
dtypes: int32(2), object(2)
memory usage: 1.6+ MB


# 1.3 Toplam satış tutarı bakımından en çok iptal edilen (azalan sıra) ürünleri lokal diske `parquet` formatında yazınız.

In [15]:
canceled_items = df_orders.filter(df_orders.orderStatus == "CANCELED")

In [16]:
joined_data = df_order_items.join(canceled_items, df_order_items.orderItemOrderId == canceled_items.orderId)

In [20]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products.csv

--2023-03-01 02:05:24--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174240 (170K) [text/plain]
Saving to: ‘/home/train/datasets/products.csv’

100%[======================================>] 174,240     --.-K/s   in 0.1s    

2023-03-01 02:05:25 (1.47 MB/s) - ‘/home/train/datasets/products.csv’ saved [174240/174240]



In [21]:
df_products = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/products.csv")

In [22]:
joined_data1 = df_products.join(joined_data, df_products.productId == joined_data.orderItemProductId)

In [23]:
joined_data1.show()

+---------+-----------------+--------------------+------------------+------------+--------------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+-------+--------------------+---------------+-----------+
|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|orderId|           orderDate|orderCustomerId|orderStatus|
+---------+-----------------+--------------------+------------------+------------+--------------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+-------+--------------------+---------------+-----------+
|      403|               18|Nike Men's CJ Eli...|              null|      129.99|http://images.acm...|          124|              50|               403|                1|           129.99|    

In [24]:
product_sales = joined_data1.groupBy("productName").agg({"orderItemSubTotal": "sum"}).withColumnRenamed("sum(orderItemSubTotal)", "productSales")

In [25]:
product_sales.show(30)

+--------------------+------------------+
|         productName|      productSales|
+--------------------+------------------+
|Field & Stream Sp...|134393.27999999968|
|Bag Boy Beverage ...| 624.7500000000001|
|Team Golf Pittsbu...|             74.97|
|Bridgestone e6 St...|            479.85|
|Titleist Pro V1x ...|            719.85|
|Merrell Women's G...|            389.97|
|TYR Boys' Team Di...| 679.8299999999999|
|Glove It Urban Br...|479.69999999999993|
|Polar Loop Activi...|             219.9|
|insta-bed Neverfl...|            149.99|
|Titleist Pro V1x ...|            287.94|
|Nike Dri-FIT Crew...|             418.0|
|LIJA Women's Eyel...|            2145.0|
|LIJA Women's Butt...|             108.0|
|Team Golf St. Lou...| 524.7900000000001|
|Stiga Master Seri...|            329.99|
|Nike Men's Dri-FI...|           65750.0|
|Perfect Fitness P...|  85785.7000000001|
|Merrell Men's All...|            109.99|
|Titleist Small Wh...|            249.99|
|Under Armour Wome...|            